In [2]:
import socket
import cv2
import numpy as np

def setup_client(ip, port):
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((ip, port))
        return client_socket
    except Exception as e:
        print(f"Error setting up client: {e}")
        raise

def send_control_message(client_socket, enable_camera=True, enable_audio=False):
    try:
        control_message = f"{enable_camera},{enable_audio}"
        client_socket.sendall(control_message.encode())
    except Exception as e:
        print(f"Error sending control message: {e}")
        raise

def receive_data(client_socket, enable_camera):
    try:
        while True:
            frame_len_bytes = client_socket.recv(4)
            if not frame_len_bytes:
                print("Error: Failed to receive frame length bytes")
                break

            frame_len = int.from_bytes(frame_len_bytes, byteorder='big')

            data = b''
            while len(data) < frame_len:
                packet = client_socket.recv(frame_len - len(data))
                if not packet:
                    print("Error: Incomplete frame data received")
                    break
                data += packet

            if enable_camera:
                if frame_len > 0:
                    frame = np.frombuffer(data, dtype=np.uint8)
                    frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)
                    if frame is not None and frame.size > 0:
                        cv2.imshow('frame', frame)
                    else:
                        print("Invalid frame data received.")
                else:
                    print("Invalid frame length received.")

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as e:
        print(f"Error receiving data: {e}")
        raise
    finally:
        client_socket.close()
        cv2.destroyAllWindows()

def main():
    ip = '172.20.10.6'
    port = 8000
    try:
        client_socket = setup_client(ip, port)
        send_control_message(client_socket)
        receive_data(client_socket, enable_camera=True)
    except KeyboardInterrupt:
        print("Keyboard interrupt detected. Exiting...")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()


Invalid frame data received.
Error receiving data: [Errno 22] Invalid argument
An unexpected error occurred: [Errno 22] Invalid argument
